In [1]:
import requests

def get_cognito_access_token(client_id, client_secret, token_url):
    # Prepare the data for the token request
    data = {
        'grant_type': 'client_credentials',
    }

    # Make the POST request to get the token
    response = requests.post(token_url, data=data, auth=(client_id, client_secret))

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        token_info = response.json()
        return token_info['access_token']
    else:
        # Handle error
        raise Exception(f"Failed to get token: {response.status_code} - {response.text}")

# Replace these with your Cognito App Client ID and Secret
client_id = '1m2f2ib1qatacreojll4aommfo'
client_secret = 'o30f697g1s7fm9jrrlveeiu4tok3aulpie90709g86gijvccc2n'

# Replace this with your Cognito token endpoint (Stg or Prod)
token_url = 'https://logiai-stg.auth.us-west-2.amazoncognito.com/oauth2/token'

access_token = get_cognito_access_token(client_id, client_secret, token_url)


Extraction the required html elements

In [ ]:
from bs4 import BeautifulSoup, Comment
import os


input_html_path = 'raw_plugines.html'
output_html_path = 'plugines.html'

def extract_important_elements(input_html_path, output_html_path):
    with open(input_html_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract key interactive elements
    elements = []
    selectors = ['input', 'button', 'select', 'textarea', 'a']
    for selector in selectors:
        elements += soup.find_all(selector)

    # Add custom class-based elements
    elements += soup.find_all(class_='test-element')

    # Create a clean, structured soup for output
    structured_soup = BeautifulSoup('<html><head><title>Extracted Elements</title></head><body></body></html>', 'html.parser')
    body = structured_soup.body

    # Add a header for organization
    body.append(structured_soup.new_tag("h1"))
    body.h1.string = "Extracted Important Elements for Testing"

    for idx, el in enumerate(elements):
        wrapper = structured_soup.new_tag("div")
        wrapper['class'] = 'extracted-element'
        comment = structured_soup.new_string(f' Element #{idx + 1}: <{el.name}> ', Comment)
        body.append(comment)
        wrapper.append(el)
        body.append(wrapper)

    with open(output_html_path, 'w', encoding='utf-8') as output_file:
        output_file.write(structured_soup.prettify())

    print(f"Structured extracted elements saved to: {output_html_path}")

# Run the function
extract_important_elements(input_html_path, output_html_path)


Structured extracted elements saved to: icons.html


Dividing the huge html into chunks

In [ ]:
import os
from bs4 import BeautifulSoup

# Paths
input_html_path = 'beta-marketPlace.html'  # Original HTML
output_folder = 'chunks_for_testing'  # Folder to save the chunks

# Function to create folder if it doesn't exist
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f" Folder '{folder_path}' created.")
    else:
        print(f" Folder '{folder_path}' already exists.")

# Function to extract meaningful testing elements and split into chunks
def extract_and_chunk_elements(input_html_path, output_folder, chunk_size=50):
    with open(input_html_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the full HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new BeautifulSoup object for the cleaned output
    new_soup = BeautifulSoup("<html><body></body></html>", 'html.parser')

    # Elements important for Selenium Testing
    important_tags = []

    # --- Form Controls ---
    important_tags += soup.find_all(['input', 'button', 'select', 'textarea'])

    # --- Links that can navigate ---
    for link in soup.find_all('a', href=True):
        if link['href'] != '#' and not link['href'].startswith('javascript'):
            important_tags.append(link)

    # --- Headings ---
    important_tags += soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

    # --- List items ---
    important_tags += soup.find_all('li')

    # --- Optional: Divs or spans that contain visible text or action ---
    for tag in soup.find_all(['div', 'span']):
        if tag.get_text(strip=True):  # Only if they have visible text
            important_tags.append(tag)

    # Create the output folder if it doesn't exist
    create_folder(output_folder)

    # Split into chunks
    total_elements = len(important_tags)
    chunk_count = (total_elements // chunk_size) + (1 if total_elements % chunk_size else 0)

    for i in range(chunk_count):
        chunk_start = i * chunk_size
        chunk_end = chunk_start + chunk_size

        # Create a new soup object for the chunk
        chunk_soup = BeautifulSoup("<html><body></body></html>", 'html.parser')

        # Append a chunk of elements to the body
        for tag in important_tags[chunk_start:chunk_end]:
            chunk_soup.body.append(tag)

        # File path for the chunk
        chunk_file_path = os.path.join(output_folder, f"beta-marketPlace_for_testing_chunk_{i+1}.html")

        # Write the chunk to a new HTML file
        with open(chunk_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(chunk_soup.prettify())
        
        print(f" Chunk {i+1} saved to: {chunk_file_path}")

# Run the extraction and chunking
extract_and_chunk_elements(input_html_path, output_folder, chunk_size=50)


✅ Folder 'chunks_for_testing' created.
✅ Chunk 1 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_1.html
✅ Chunk 2 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_2.html
✅ Chunk 3 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_3.html
✅ Chunk 4 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_4.html
✅ Chunk 5 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_5.html
✅ Chunk 6 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_6.html
✅ Chunk 7 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_7.html
✅ Chunk 8 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_8.html
✅ Chunk 9 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_9.html
✅ Chunk 10 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_10.html
✅ Chunk 11 saved to: chunks_for_testing\beta-marketPlace_for_testing_chunk_11.html
✅ Chunk 12 saved to: chunks_for_testing\beta-marketPlace_for_testi

Generating File Structure 

In [ ]:
import os

# Define the folder structure
folder_structure = {
    "selenium_pytest_project": {
        "configurations": [],
        "page_objects": [],
        "test_cases": [],
        "test_data": [],
        "utilities": [],
        "reports": []
    }
}

# Function to create folders
def create_folders(base_path, folder_structure):
    for folder, subfolders in folder_structure.items():
        folder_path = os.path.join(base_path, folder)
        
        # Create the main project folder
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f" Created folder: {folder_path}")
        
        # Create subfolders inside each main folder
        for subfolder in subfolders:
            subfolder_path = os.path.join(folder_path, subfolder)
            if not os.path.exists(subfolder_path):
                os.makedirs(subfolder_path)
                print(f" Created subfolder: {subfolder_path}")

# Base path where the project folder will be created
base_path = os.getcwd()  # Current working directory

# Create the folder structure
create_folders(base_path, folder_structure)

print(" Folder structure created successfully.")


✅ Created folder: c:\genai\ai_for_automation\selenium_pytest_project
✅ Created subfolder: c:\genai\ai_for_automation\selenium_pytest_project\configurations
✅ Created subfolder: c:\genai\ai_for_automation\selenium_pytest_project\page_objects
✅ Created subfolder: c:\genai\ai_for_automation\selenium_pytest_project\test_cases
✅ Created subfolder: c:\genai\ai_for_automation\selenium_pytest_project\test_data
✅ Created subfolder: c:\genai\ai_for_automation\selenium_pytest_project\utilities
✅ Created subfolder: c:\genai\ai_for_automation\selenium_pytest_project\reports
✅ Folder structure created successfully.


General conftest

In [1]:
import openai
import os

# Function to extract code from the OpenAI response text
def extract_code(response_text):
    code_blocks = response_text.split("```python")
    if len(code_blocks) > 1:
        return code_blocks[1].split("```")[0].strip()
    return response_text.strip()

# Function to interact with the OpenAI API to generate conftest.py
def generate_conftest_with_openai(url, output_folder):
    # Prepare the configurations folder path
    conftest_folder_path = os.path.join(output_folder, "configurations")
    
    # Make sure folder exists
    os.makedirs(conftest_folder_path, exist_ok=True)

    # Prepare a short + light prompt (no chunks, only instructions)
    prompt = f"""You are an expert in Selenium + Pytest automation framework.

Please generate a clean, production-ready `conftest.py` file using the following best practices:

**Driver Fixture (`driver`)**:
- Use `webdriver_manager` to automatically manage ChromeDriver.
- Configure Chrome options with the following:
  - `--start-maximized`
  - `--disable-notifications`
  - `--disable-infobars`
  - `--disable-extensions`
  - `--no-sandbox`
- Enable headless mode if the environment variable `HEADLESS=true` is set.
- Set `implicitly_wait(10)` for the WebDriver instance.
- Use logging to indicate driver setup and teardown.
- Use `scope="session"` to initialize the WebDriver **only once** per Pytest run.

**Base URL Fixture (`base_url`)**:
- Use `scope="session"` to provide a consistent base URL throughout the session.
- Default to: {url}
- Allow overriding via `BASE_URL` environment variable.

**General Requirements**:
- Follow Pytest conventions.
- Include useful comments above each fixture explaining its role.
- Provide a small example in comments demonstrating how to use both `driver` and `base_url` in a test.

Output only the Python code in a proper ```python``` code block.
"""


    client = openai.OpenAI(
        base_url="https://stg-logiai-service.np.logitech.io/openai/v1",
        api_key=access_token  # Your correct access_token
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            max_tokens=8000,
        )

        # Extract and clean code
        conftest_code = extract_code(completion.choices[0].message.content)

        # Save the code
        conftest_path = os.path.join(conftest_folder_path, "conftest.py")
        with open(conftest_path, 'w', encoding='utf-8') as file:
            file.write(conftest_code)

        print(f" conftest.py created successfully at {conftest_path}")

    except Exception as e:
        print(f" Error occurred while generating conftest.py: {e}")

# === Main Run ===
# Paths
output_folder = r"C:\genai\ai_for_automation\selenium_pytest_project\configurations"

# URL to test
url = "https://beta-marketplace.logitech.com/"

# Run function
generate_conftest_with_openai(url, output_folder)


NameError: name 'access_token' is not defined

POM Chunks

In [ ]:
import openai
import os

# Function to extract code from OpenAI response
def extract_code(response_text):
    code_blocks = response_text.split("```python")
    if len(code_blocks) > 1:
        return code_blocks[1].split("```")[0].strip()
    return response_text.strip()

# Function to read all chunk files from a folder
def read_chunks_from_folder(chunks_folder):
    chunk_files = []
    for filename in os.listdir(chunks_folder):
        file_path = os.path.join(chunks_folder, filename)
        if filename.endswith(".html") or filename.endswith(".txt"):
            with open(file_path, 'r', encoding='utf-8') as file:
                chunk_content = file.read()
                chunk_files.append((filename, chunk_content))
    return chunk_files

# Function to generate Page Object files
def generate_pom_from_chunks(project_folder, chunks_folder):
    # Define page_objects folder path
    page_objects_folder = os.path.join(project_folder, "page_objects")
    os.makedirs(page_objects_folder, exist_ok=True)

    # Read all chunks
    chunks = read_chunks_from_folder(chunks_folder)
    if not chunks:
        print(f" No valid HTML chunk files found in {chunks_folder}")
        return

    # OpenAI client setup
    client = openai.OpenAI(
        base_url="https://stg-logiai-service.np.logitech.io/openai/v1",
        api_key=access_token  # Make sure this is already available
    )

    for filename, chunk_content in chunks:
        # Create a simple prompt for each chunk
        prompt = f"""
You are an expert in building Selenium Page Object Model (POM) classes.

Below is an HTML snippet:
{chunk_content}

Based on this HTML, generate a Python POM class:
- Class name should be based on the file name (without extension), in PascalCase.
- Use Selenium locators and `By`.
- Include an `__init__` method with a WebDriver instance.
- For each interactive element (like buttons), define methods with:
  - WebDriverWait and expected_conditions for reliable interaction
  - Scroll into view using JavaScript before clicking
  - Retry logic for StaleElementReferenceException
  - Fallback click using ActionChains or JavaScript if normal click fails
- Follow production-level coding standards and error handling
- Do not include test cases
- Wrap the final code in a Python code block
"""


        try:
            completion = client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                stream=False,
                max_tokens=1200,
            )

            # Extract the code
            pom_code = extract_code(completion.choices[0].message.content)

            # Convert filename to class name
            base_name = os.path.splitext(filename)[0]
            class_name = "".join(word.capitalize() for word in base_name.split("_"))

            # Save file
            pom_file_path = os.path.join(page_objects_folder, f"{base_name}.py")
            with open(pom_file_path, 'w', encoding='utf-8') as file:
                file.write(pom_code)

            print(f" Created Page Object: {pom_file_path}")

        except Exception as e:
            print(f" Error processing {filename}: {e}")

# === Main Run ===
# Input paths
waste_folder = r"C:\genai\ai_for_automation\wastage\pom_chunks"
chunks_folder = r"C:\genai\ai_for_automation\chunks_for_testing"

# Run function
generate_pom_from_chunks(waste_folder, chunks_folder)


✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_1.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_10.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_11.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_12.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_13.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_14.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_15.py
✅ Created Page Object: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\beta-marketPlace_for_testing_chunk_2.py
✅ Created Page Obj

In [ ]:
import os

def combine_chunks_as_is(folder_path, output_file_path):
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            for filename in sorted(os.listdir(folder_path)):
                file_path = os.path.join(folder_path, filename)
                if os.path.isfile(file_path) and filename.endswith('.py'):
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                        output_file.write(f"# ----- {filename} -----\n")
                        output_file.write(content + "\n\n")
        print(f" Combined file created at: {output_file_path}")
    except Exception as e:
        print(f" Error: {e}")

# Example usage
folder_path = r"C:\genai\ai_for_automation\wastage\pom_chunks\page_objects"
output_file_path = r"C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\page_objects_combined.py"

combine_chunks_as_is(folder_path, output_file_path)


✅ Combined file created at: C:\genai\ai_for_automation\wastage\pom_chunks\page_objects\page_objects_combined.py


combined output for page object chunks (input file)

In [ ]:
import openai
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def read_single_file(file_path):
    """Reads and returns the content of a single file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        logger.error(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    """Extracts Python code from markdown-style response.""" 
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm(file_content, api_key, base_url):
    """Sends a single file's content to the LLM and returns optimized Python code."""
    client = openai.OpenAI(
        base_url=base_url,
        api_key=api_key
    )

    prompt = f"""
You are a Selenium expert using Pytest with the Page Object Model (POM).

You will be given a raw Page Object class file. Please do the following:

1. Do NOT change any of the existing locators (e.g., self.username_input = ...).
2. Do NOT rename or merge any methods.
3. Ensure that all functions are retained and NO duplicates exist.
4. Clean the code, maintain structure, and preserve the original functionality.

IMPORTANT:
- Keep all By locators and element actions intact.
- The final code should contain all methods from the original, even if they're similar.
- Remove duplicate method definitions (same method name more than once).
- Preserve method order and structure.

Now clean and return the corrected class file for the following code:

{file_content}
"""
    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            max_tokens=9000,
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        logger.error(f"Error while sending content to LLM: {e}")
        return ""

def create_output_file(output_file_path, optimized_content):
    """Writes optimized code to output file."""
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(optimized_content)
        logger.info(f" Optimized file created at {output_file_path}")
    except Exception as e:
        logger.error(f"Error writing to output file: {e}")

def process_single_file(input_file_path, output_file_path, api_key, base_url):
    """Main function for processing a single file."""
    file_content = read_single_file(input_file_path)
    if not file_content:
        logger.warning(f"No content found in file: {input_file_path}")
        return

    optimized_content = send_to_llm(file_content, api_key, base_url)
    if not optimized_content:
        logger.warning("No optimized content received from LLM.")
        return

    create_output_file(output_file_path, optimized_content)

# Example usage
input_file_path = r"C:\genai\ai_for_automation\wastage\page_objects_combined.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects.py"
api_key = access_token  # Make sure access_token is defined or imported
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

process_single_file(input_file_path, output_file_path, api_key, base_url)


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Optimized file created at C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects.py


Combining the chunks of pom (input folder)

In [ ]:
import openai
import os

def read_all_chunks_from_folder(folder_path):
    """Reads and combines all .py files from the given folder."""
    combined_content = ""
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".py"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
                combined_content += f.read() + "\n\n"
    return combined_content

def extract_code(response_text):
    """Extracts Python code from markdown-style response.""" 
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm(file_content, api_key, base_url):
    """Sends the combined content to the LLM and returns optimized code."""
    client = openai.OpenAI(
        base_url=base_url,
        api_key=api_key
    )

    prompt = (
        "You are a Python Selenium expert following the Page Object Model (POM).\n"
        "Below is a class split across chunks. Please:\n"
        "- Combine all chunks into one clean class.\n"
        "- Keep ALL functions (do not remove any).\n"
        "- Do NOT change any locators.\n"
        "- Remove duplicate function names.\n"
        "- Ensure class is clean and well-structured.\n"
        "- go through every function if there is any logic mistake try to write a comment after the function. \n"
        "- try to create all the function in one single class"
        f"{file_content}"
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            max_tokens=9000,
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"Error from LLM: {e}")
        return ""

def create_output_file(output_file_path, optimized_content):
    """Writes optimized code to the specified output file."""
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(optimized_content)
        print(f" Optimized file created at {output_file_path}")
    except Exception as e:
        print(f"Error writing to file: {e}")

def process_folder(folder_path, output_file_path, api_key, base_url):
    """Reads all chunk files, combines them, sends to LLM, and writes result."""
    file_content = read_all_chunks_from_folder(folder_path)
    if not file_content:
        print(f"No Python files found in: {folder_path}")
        return

    optimized_content = send_to_llm(file_content, api_key, base_url)
    if not optimized_content:
        print("No optimized content returned.")
        return

    create_output_file(output_file_path, optimized_content)

# Example usage
input_folder = r"C:\genai\ai_for_automation\wastage\pom_chunks\page_objects"
output_file = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\try_page_objects.py"
api_key = access_token  # Replace or define this
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

process_folder(input_folder, output_file, api_key, base_url)


✅ Optimized file created at C:\genai\ai_for_automation\selenium_pytest_project\page_objects\try_page_objects.py


In [ ]:
import openai
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def read_single_file(file_path):
    """Reads and returns the content of a single file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        logger.error(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    """Extracts Python code from markdown-style response.""" 
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm(file_content, api_key, base_url):
    """Sends a single file's content to the LLM and returns optimized Python code."""
    client = openai.OpenAI(
        base_url=base_url,
        api_key=api_key
    )

    prompt = f"""
You are an expert in Selenium and Pytest automation.

You will be provided with a raw test case class file. Your task is to:

1. **Do NOT modify** any existing locators (e.g., self.username_input = ...).
2. **Do NOT rename or merge** any methods.
3. **Ensure that all methods are kept** without duplication. If any method is duplicated, remove the duplicates.
4. **Preserve the structure and functionality** of the original code, ensuring that all methods and logic are intact.
5. **Clean the code**, removing unnecessary or redundant lines, but maintaining clarity and readability.

IMPORTANT:
- Keep all By locators and element actions as they are, without any changes.
- Ensure the final output contains **all methods** from the original file, even if they appear similar.
- **Remove duplicate methods** (same method name, same functionality, appearing more than once).
- **Preserve the method order** and **class structure** exactly as in the original file.

Return the corrected class file, maintaining the original functionality, for the following input code:

{file_content}
"""

    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            max_tokens=9000,
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        logger.error(f"Error while sending content to LLM: {e}")
        return ""

def create_output_file(output_file_path, optimized_content):
    """Writes optimized code to output file."""
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(optimized_content)
        logger.info(f" Optimized file created at {output_file_path}")
    except Exception as e:
        logger.error(f"Error writing to output file: {e}")

def process_single_file(input_file_path, output_file_path, api_key, base_url):
    """Main function for processing a single file."""
    file_content = read_single_file(input_file_path)
    if not file_content:
        logger.warning(f"No content found in file: {input_file_path}")
        return

    optimized_content = send_to_llm(file_content, api_key, base_url)
    if not optimized_content:
        logger.warning("No optimized content received from LLM.")
        return

    create_output_file(output_file_path, optimized_content)

# Example usage
input_file_path = r"C:\genai\ai_for_automation\wastage\page_objects_combined.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects.py"
api_key = access_token  # Make sure access_token is defined or imported
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

process_single_file(input_file_path, output_file_path, api_key, base_url)


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Optimized file created at C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects.py


Chunks for test cases

In [7]:
import openai
import os

# === Utility Functions ===

def read_files_from_folder(folder_path):
    contents = []
    try:
        for filename in sorted(os.listdir(folder_path)):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    contents.append(file.read())
        return contents
    except Exception as e:
        print(f"Error reading files from folder: {e}")
        return []

def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm_single_chunk(html_chunk, pom_code, conftest_code, api_key, base_url):
    from openai import OpenAI
    client = OpenAI(api_key=api_key, base_url=base_url)

    prompt = (
        "You are an expert in Selenium and Pytest.\n"
        "Generate a Python test function using pytest and the provided conftest fixture and Page Object Model (POM).\n"
        "Use correct imports.\n\n"
        f"HTML Snippet:\n{html_chunk}\n\n"
        f"POM Code:\n{pom_code}\n\n"
        f"conftest.py:\n{conftest_code}"
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=4096
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"LLM call failed: {e}")
        return ""



def save_combined_test(output_file_path, test_code):
    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(test_code)
        print(f"Final test file saved at: {output_file_path}")
    except Exception as e:
        print(f"Error saving final test file: {e}")

# === Main Function ===

def generate_single_test_file(html_folder, pom_file_path, conftest_path, output_file_path, api_key, base_url):
    html_chunks = read_files_from_folder(html_folder)
    pom_code = read_file(pom_file_path)
    conftest_code = read_file(conftest_path)

    combined_test_code = ""

    for i, chunk in enumerate(html_chunks):
        print(f"\n--- Processing chunk {i+1}/{len(html_chunks)} ---")
        test_code = send_to_llm_single_chunk(chunk, pom_code, conftest_code, api_key, base_url)
        if test_code:
            combined_test_code += f"# === Test from chunk {i+1} ===\n{test_code}\n\n"
        else:
            print(f"No test code received for chunk {i+1}.")

    if combined_test_code:
        save_combined_test(output_file_path, combined_test_code)
    else:
        print("No test code generated.")


# === Update these with correct paths ===

html_folder = r"C:\genai\ai_for_automation\chunks_for_testing"
pom_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects.py"  # Use correct single POM file
conftest_path = r"C:\genai\ai_for_automation\selenium_pytest_project\configurations\configurations\conftest.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\try_test_final_combined.py"
api_key = access_token
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

generate_single_test_file(html_folder, pom_file_path, conftest_path, output_file_path, api_key, base_url)



--- Processing chunk 1/15 ---

--- Processing chunk 2/15 ---

--- Processing chunk 3/15 ---

--- Processing chunk 4/15 ---

--- Processing chunk 5/15 ---

--- Processing chunk 6/15 ---

--- Processing chunk 7/15 ---

--- Processing chunk 8/15 ---

--- Processing chunk 9/15 ---

--- Processing chunk 10/15 ---

--- Processing chunk 11/15 ---

--- Processing chunk 12/15 ---

--- Processing chunk 13/15 ---

--- Processing chunk 14/15 ---

--- Processing chunk 15/15 ---
Final test file saved at: C:\genai\ai_for_automation\selenium_pytest_project\test_cases\try_test_final_combined.py


In [ ]:
prompt = f"""
You are a Selenium expert using Pytest.

You will be given a merged raw test case file composed of multiple test chunks. Your task is to:

1. Consolidate the code into a **single clean test file**.
2. **Do NOT remove or rename any test functions** — all test logic must be preserved.
3. **Eliminate duplicate imports** and organize them properly.
4. Remove **duplicate test cases** (same name and logic).
5. Ensure that **each test function is unique**, well-structured, and properly formatted.
6. Clean and organize fixtures and helper methods without changing their behavior.

IMPORTANT:
- **Preserve test logic and data** from all chunks.
- Ensure **all test cases and fixtures are retained** unless they are exact duplicates.
- Adjust **import paths consistently** (e.g., unify `from POM` vs `from pom` vs `from plugin_marketplace_page`).
- Ensure **no syntax errors** and that the file is ready to run with `pytest`.

Return a single, clean, final test file with all necessary test functions and imports.

Here is the raw test file:

{file_content}
"""


###final test cases production  

In [ ]:
import openai
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def read_single_file(file_path):
    """Reads and returns the content of a single file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        logger.error(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    """Extracts Python code from markdown-style response.""" 
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm(file_content, api_key, base_url):
    """Sends a single file's content to the LLM and returns optimized Python code."""
    client = openai.OpenAI(
        base_url=base_url,
        api_key=api_key
    )

    prompt = f"""
You are a Selenium expert using Pytest.

You will be given a merged raw test case file composed of multiple test chunks. Your task is to:

1. Consolidate the code into a **single clean test file**.
2. **Do NOT remove or rename any test functions** — all test logic must be preserved.
3. **Eliminate duplicate imports** and organize them properly.
4. Remove **duplicate test cases** (same name and logic).
5. Ensure that **each test function is unique**, well-structured, and properly formatted.
6. Clean and organize fixtures and helper methods without changing their behavior.

IMPORTANT:
- **Preserve test logic and data** from all chunks.
- Ensure **all test cases and fixtures are retained** unless they are exact duplicates.
- Adjust **import paths consistently** (e.g., unify `from POM` vs `from pom` vs `from plugin_marketplace_page`).
- Ensure **no syntax errors** and that the file is ready to run with `pytest`.

Return a single, clean, final test file with all necessary test functions and imports.

Here is the raw test file:

{file_content}
"""
    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
            max_tokens=9000,
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        logger.error(f"Error while sending content to LLM: {e}")
        return ""

def create_output_file(output_file_path, optimized_content):
    """Writes optimized code to output file."""
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(optimized_content)
        logger.info(f" Optimized file created at {output_file_path}")
    except Exception as e:
        logger.error(f"Error writing to output file: {e}")

def process_single_file(input_file_path, output_file_path, api_key, base_url):
    """Main function for processing a single file."""
    file_content = read_single_file(input_file_path)
    if not file_content:
        logger.warning(f"No content found in file: {input_file_path}")
        return

    optimized_content = send_to_llm(file_content, api_key, base_url)
    if not optimized_content:
        logger.warning("No optimized content received from LLM.")
        return

    create_output_file(output_file_path, optimized_content)

# Example usage
input_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\try_test_final_combined.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\try_test_case.py"
api_key = access_token  # Make sure access_token is defined or imported
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

process_single_file(input_file_path, output_file_path, api_key, base_url)


INFO:httpx:HTTP Request: POST https://stg-logiai-service.np.logitech.io/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Optimized file created at C:\genai\ai_for_automation\selenium_pytest_project\test_cases\try_test_case.py


scrape (pom, test data)

In [ ]:
import json
from pathlib import Path

# Input paths
pom_path = Path("C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objects.py")
test_data_path = Path("C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case.py")

# Output path
output_path = Path("output/test_install_button.py")
output_path.parent.mkdir(parents=True, exist_ok=True)


def load_json(path):
    with open(path, "r") as f:
        return json.load(f)


def generate_pom_class(pom):
    class_name = pom["class_name"]
    elements = pom["elements"]
    actions = pom["actions"]

    lines = [
        f"class {class_name}:",
        f"    def __init__(self, driver):",
        f"        self.driver = driver",
        ""
    ]

    for el in elements:
        if "{app_id}" in el["value"]:
            line = f"    def get_{el['name']}(self, app_id):"
            val = el["value"].replace("{app_id}", "' + app_id + '")
            body = f"        return self.driver.find_element({el['by']}, f\"{val}\")"
        else:
            line = f"    def get_{el['name']}(self):"
            body = f"        return self.driver.find_element({el['by']}, \"{el['value']}\")"
        lines += [line, body, ""]

    for action in actions:
        name = action["name"]
        el = action["element"]
        params = ", ".join(["self"] + action.get("params", []))
        call_args = ", ".join(action.get("params", []))
        lines += [
            f"    def {name}({params}):",
            f"        self.get_{el}({call_args}).click()",
            ""
        ]

    return "\n".join(lines)


def generate_test_case(test, pom_class_name, url_suffix):
    return f"""
def {test['test_name']}(driver, base_url):
    driver.get(base_url + "{url_suffix}")
    page = {pom_class_name}(driver)

    app_id = "{test['app_id']}"
    page.click_install_button(app_id)

    confirmation = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.install-confirmation"))
    )
    assert "{test['expected_text']}" in confirmation.text, "Installation confirmation not found."
"""


def generate_combined_script(pom_data, test_data):
    imports = """from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pytest
"""

    pom_code = generate_pom_class(pom_data)
    test_code = generate_test_case(test_data, pom_data["class_name"], pom_data["url_suffix"])
    return imports + "\n\n" + pom_code + "\n" + test_code


# Run generation
pom_data = load_json(pom_path)
test_data = load_json(test_data_path)
code = generate_combined_script(pom_data, test_data)

# Write output
output_path.write_text(code)
print(f" Generated combined POM + test case: {output_path}")


OSError: [Errno 22] Invalid argument: 'C:\\genai\x07i_for_automation\\selenium_pytest_project\\page_objects\\page_objects.py'

Single test cases from chunks

In [ ]:
import openai
import os

# === Utility Functions ===

def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm_single_chunk(html_combined, pom_code, conftest_code, api_key, base_url):
    from openai import OpenAI
    client = OpenAI(api_key=api_key, base_url=base_url)

    prompt = (
        "You are an expert in Selenium and Pytest.\n"
        "Generate a single Python test file using pytest, based on the HTML snippet below.\n"
        "Use Page Object Model (POM) functions and conftest fixtures properly.\n"
        "Ensure correct and clean imports.\n"
        "Do not repeat POM or conftest code.\n\n"
        f"HTML Snippet:\n{html_combined}\n\n"
        f"POM Code:\n{pom_code}\n\n"
        f"conftest.py:\n{conftest_code}"
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=4096
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"LLM call failed: {e}")
        return ""

def save_combined_test(output_file_path, test_code):
    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(test_code)
        print(f"Final test file saved at: {output_file_path}")
    except Exception as e:
        print(f"Error saving final test file: {e}")

# === Main Function ===

def generate_test_from_single_html_file(html_file_path, pom_file_path, conftest_path, output_file_path, api_key, base_url):
    html_combined = read_file(html_file_path)
    pom_code = read_file(pom_file_path)
    conftest_code = read_file(conftest_path)

    print("Sending single combined HTML chunk to LLM...")

    test_code = send_to_llm_single_chunk(html_combined, pom_code, conftest_code, api_key, base_url)

    if test_code:
        save_combined_test(output_file_path, test_code)
    else:
        print("No test code generated.")

# === Update these with correct paths ===

html_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_combined.py"
pom_file_path = r"C:\genai\AI_for_automation_stage _1.1\selenium_pytest_project\page_objects\combined_output.py"
conftest_path = r"C:\genai\AI_for_automation_stage _1.1\selenium_pytest_project\configurations\conftest.py"
output_file_path = r"C:\genai\AI_for_automation_stage _1.1\selenium_pytest_project\test_cases\test_final_combined.py"
api_key = access_token
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

generate_test_from_single_html_file(html_file_path, pom_file_path, conftest_path, output_file_path, api_key, base_url)


Sending single combined HTML chunk to LLM...
Final test file saved at: C:\genai\AI_for_automation_stage _1.1\selenium_pytest_project\test_cases\test_final_combined.py


In [ ]:
import os
from openai import OpenAI

# === Utility Functions ===

def read_files_from_folder(folder_path):
    contents = []
    try:
        for filename in sorted(os.listdir(folder_path)):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    contents.append(file.read())
        return "\n\n".join(contents)
    except Exception as e:
        print(f"Error reading files from folder: {e}")
        return ""

def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def send_to_llm_merge_and_optimize(html_combined, pom_code, conftest_code, raw_test_code, api_key, base_url):
    client = OpenAI(api_key=api_key, base_url=base_url)

    prompt = (
        "You are a senior QA automation engineer.\n"
        "Your task is to optimize and clean up the following raw test functions written using Pytest and Selenium.\n"
        "Use the provided Page Object Model (POM) and conftest.py fixture correctly.\n"
        "Fix duplicate imports, incorrect structure, and ensure the output is ONE clean test file using POM functions and fixtures.\n"
        "Only output the final Python test file.\n\n"
        f"HTML Chunks (merged):\n{html_combined}\n\n"
        f"POM Code:\n{pom_code}\n\n"
        f"conftest.py:\n{conftest_code}\n\n"
        f"Raw Combined Test Code:\n{raw_test_code}"
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=4096
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"LLM call failed: {e}")
        return ""

def save_final_test(output_file_path, test_code):
    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(test_code)
        print(f" Final test file saved at: {output_file_path}")
    except Exception as e:
        print(f" Error saving final test file: {e}")

# === Main Function ===

def refine_combined_test_case(html_folder, pom_file_path, conftest_path, raw_test_path, output_file_path, api_key, base_url):
    html_combined = read_files_from_folder(html_folder)
    pom_code = read_file(pom_file_path)
    conftest_code = read_file(conftest_path)
    raw_test_code = read_file(raw_test_path)

    final_test_code = send_to_llm_merge_and_optimize(html_combined, pom_code, conftest_code, raw_test_code, api_key, base_url)

    if final_test_code:
        save_final_test(output_file_path, final_test_code)
    else:
        print(" No test code generated.")

# === Update with actual paths ===

html_folder = r"C:\genai\ai_for_automation\chunks_for_testing"
pom_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objest.py"
conftest_path = r"C:\genai\ai_for_automation\selenium_pytest_project\configurations\configurations\conftest.py"
raw_test_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_combined.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_cleaned.py"
api_key = access_token
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

refine_combined_test_case(html_folder, pom_file_path, conftest_path, raw_test_path, output_file_path, api_key, base_url)


LLM call failed: Error code: 400 - {'detail': 'Error code: 400 - {\'error\': {\'message\': "This model\'s maximum context length is 128000 tokens. However, your messages resulted in 132501 tokens. Please reduce the length of the messages.", \'type\': \'invalid_request_error\', \'param\': \'messages\', \'code\': \'context_length_exceeded\'}}'}
⚠️ No test code generated.


In [24]:
import openai
import os

# === Utility Functions ===

def read_files_from_folder(folder_path):
    contents = []
    try:
        for filename in sorted(os.listdir(folder_path)):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    contents.append(file.read())
        return contents
    except Exception as e:
        print(f"Error reading files from folder: {e}")
        return []

def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def chunk_text(text, max_chars=10000):  # Split into 10,000 characters per chunk
    """
    Split text into substrings of up to max_chars.
    """
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]

def send_to_llm_single_chunk(html_chunk, pom_code, conftest_code, api_key, base_url):
    from openai import OpenAI
    client = OpenAI(api_key=api_key, base_url=base_url)

    prompt = (
        "You are an expert in Selenium and Pytest.\n"
        "Generate a Python test function using pytest and the provided conftest fixture and Page Object Model (POM).\n"
        "Use correct imports.\n\n"
        f"HTML Snippet:\n{html_chunk}\n\n"
        f"POM Code:\n{pom_code}\n\n"
        f"conftest.py:\n{conftest_code}"
    )

    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=4096
        )
        return extract_code(completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"LLM call failed: {e}")
        return ""

def save_combined_test(output_file_path, test_code):
    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(test_code)
        print(f"Final test file saved at: {output_file_path}")
    except Exception as e:
        print(f"Error saving final test file: {e}")

# === Main Function ===

def generate_single_test_file(html_folder, pom_file_path, conftest_path, raw_test_path, output_file_path, api_key, base_url):
    # Read HTML chunks
    html_chunks = read_files_from_folder(html_folder)
    html_combined = "\n\n".join(html_chunks)
    # Read POM and conftest
    pom_code = read_file(pom_file_path)
    conftest_code = read_file(conftest_path)

    # Chunk the combined HTML
    html_chunks_split = chunk_text(html_combined, max_chars=10000)

    combined_test_code = ""

    # Process each chunk and generate test code
    for i, chunk in enumerate(html_chunks_split):
        print(f"\n--- Processing chunk {i+1}/{len(html_chunks_split)} ---")
        test_code = send_to_llm_single_chunk(chunk, pom_code, conftest_code, api_key, base_url)
        if test_code:
            combined_test_code += f"# === Test from chunk {i+1} ===\n{test_code}\n\n"
        else:
            print(f"No test code received for chunk {i+1}.")

    if combined_test_code:
        save_combined_test(output_file_path, combined_test_code)
    else:
        print("No test code generated.")

# === Update these with correct paths ===

html_folder = r"C:\genai\ai_for_automation\chunks_for_testing"
pom_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objest.py"
conftest_path = r"C:\genai\ai_for_automation\selenium_pytest_project\configurations\configurations\conftest.py"
raw_test_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_combined.py"
output_file_path = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_cleaned.py"
api_key = access_token  # Insert your API key
base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

# Call to generate the final test code
generate_single_test_file(html_folder, pom_file_path, conftest_path, raw_test_path, output_file_path, api_key, base_url)



--- Processing chunk 1/41 ---
LLM call failed: Error code: 401 - {'detail': 'Invalid Credential'}
No test code received for chunk 1.

--- Processing chunk 2/41 ---
LLM call failed: Error code: 401 - {'detail': 'Invalid Credential'}
No test code received for chunk 2.

--- Processing chunk 3/41 ---
LLM call failed: Error code: 401 - {'detail': 'Invalid Credential'}
No test code received for chunk 3.

--- Processing chunk 4/41 ---
LLM call failed: Error code: 401 - {'detail': 'Invalid Credential'}
No test code received for chunk 4.

--- Processing chunk 5/41 ---
LLM call failed: Error code: 401 - {'detail': 'Invalid Credential'}
No test code received for chunk 5.

--- Processing chunk 6/41 ---
LLM call failed: Error code: 401 - {'detail': 'Invalid Credential'}
No test code received for chunk 6.

--- Processing chunk 7/41 ---
LLM call failed: Error code: 401 - {'detail': 'Invalid Credential'}
No test code received for chunk 7.

--- Processing chunk 8/41 ---
LLM call failed: Error code: 40

KeyboardInterrupt: 

In [ ]:
import openai
import os

def read_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"❌ Error reading file {file_path}: {e}")
        return ""

def extract_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

def optimize_code_with_llm(code, api_key, base_url):
    client = openai.OpenAI(api_key=api_key, base_url=base_url)

    prompt = (
        "You are a senior Python engineer.\n"
        "Optimize the following code by:\n"
        "- Reducing redundancy\n"
        "- Refactoring repeated patterns\n"
        "- Improving readability and structure\n"
        "- Keeping functionality exactly the same\n\n"
        f"```python\n{code}\n```"
    )

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2048
        )
        return extract_code(response.choices[0].message.content.strip())
    except Exception as e:
        print(f" LLM optimization failed: {e}")
        return ""

def write_file(file_path, content):
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f" Optimized code saved to: {file_path}")
    except Exception as e:
        print(f" Error writing file: {e}")

# === MAIN RUN ===

if __name__ == "__main__":
    input_file = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_combined.py"
    output_file = r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case.py"
    api_key = access_token  # Replace with actual token or use env var
    base_url = "https://stg-logiai-service.np.logitech.io/openai/v1"

    code = read_file(input_file)
    if code:
        optimized = optimize_code_with_llm(code, api_key, base_url)
        if optimized:
            write_file(output_file, optimized)
        else:
            print(" No optimized code returned.")


✅ Optimized code saved to: C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case.py


In [ ]:
import openai
import os

# === File Handling Helpers ===

def read_file(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f" Error reading {path}: {e}")
        return ""

def write_file(path, content):
    try:
        with open(path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f" Optimized test cases saved to: {path}")
    except Exception as e:
        print(f" Error writing to {path}: {e}")

# === Extract Python Code From Markdown ===

def extract_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    return response_text.strip()

# === LLM Optimization Call ===

def optimize_test_code(test_code, pom_code, conftest_code, api_key, base_url):
    client = openai.OpenAI(api_key=api_key, base_url=base_url)

    prompt = (
        "You are a senior Python QA engineer using Selenium + Pytest.\n"
        "Here is the test case file I generated. Optimize it by:\n"
        "- Refactoring duplicate or verbose code\n"
        "- Using helper functions from POM\n"
        "- Utilizing fixtures from conftest.py\n"
        "- Maintaining the same behavior and logic\n\n"
        f"--- conftest.py ---\n{conftest_code}\n\n"
        f"--- POM ---\n{pom_code}\n\n"
        f"--- Test Cases ---\n```python\n{test_code}\n```"
    )

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2048,
            temperature=0.3,
        )
        return extract_code(response.choices[0].message.content.strip())
    except Exception as e:
        print(f" LLM request failed: {e}")
        return ""

# === Main Runner ===

def optimize_tests(test_file, pom_file, conftest_file, output_file, api_key, base_url):
    test_code = read_file(test_file)
    pom_code = read_file(pom_file)
    conftest_code = read_file(conftest_file)

    if not test_code:
        print(" Test case file is empty.")
        return

    print("📤 Sending test cases + context to LLM...")
    optimized = optimize_test_code(test_code, pom_code, conftest_code, api_key, base_url)

    if optimized:
        write_file(output_file, optimized)
    else:
        print(" No optimized code returned.")

# === Example Execution ===

if __name__ == "__main__":
    optimize_tests(
        test_file=r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_final_combined.py",
        pom_file=r"C:\genai\ai_for_automation\selenium_pytest_project\page_objects\page_objest.py",
        conftest_file=r"C:\genai\ai_for_automation\selenium_pytest_project\configurations\configurations\conftest.py",
        output_file=r"C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case.py",
        api_key=access_token,
        base_url="https://stg-logiai-service.np.logitech.io/openai/v1"
    )


📤 Sending test cases + context to LLM...
✅ Optimized test cases saved to: C:\genai\ai_for_automation\selenium_pytest_project\test_cases\test_case.py
